Installing the requirements

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets
!pip install huggingface-hub
!pip install nltk

Importing the necessary libraries 

In [ ]:
import nltk 
import random 
import logging 
import tensorflow as tf 
from tensorflow import keras 

nltk.download('punkt')
# chỉ ghi lại thông báo lỗi 
tf.get_logger().setLevel(logging.ERROR)
# Set random Seed 
tf.keras.utils.set_random_seed(42)


Define certain variables 

In [ ]:
TOKENIZER_BATCH_SIZE = 256 # Kích thước hàng loạt để đào tạo mã thôngh báo trên 
TOKENIZER_VOCABULARY = 25000 # TỔNG SỐ PHỤ TỪ DUY NHẤT MÀ TOKENIZER CÓ THỂ CÓ 

BLOCK_SIZE = 128 # SỐ LƯỢNG TOKENS TỐI ĐA CỦA MỘT BỘ MẪU ĐẦU VÀO 
NSP_PROB = 0.50 # XÁC XUẤT SINH RA CÂU TIẾP THEO THỰC TẾ TRONG NSP 
SHORT_SEQ_PROB = 0.1 # XÁC XUẤT XINH RA CHUỖI NGẮN HƠN ĐỂ GIẢM THIỂU SỰ KHÔNG KHỚP 
# GIỮA TIỀN HUẤN LUYỆN VÀ TINH CHỈNH 
MAX_LENGTH = 512 # SỐ LƯỢNG TỐI ĐA MẪU TOKENS ĐẦU VÀO SAU KHI ĐỆM 

MLM_PROB = 0.2  # XÁC XUẤT VỚI NHỮNG TOKENS ĐƯỢC CHE TRONG MASKED LANGUAGE MODEL 

TRAIN_BATCH_SIZE = 2 # KÍCH THƯỚC HÀNG LOẠT CHO HUẤN LUYỆN TRƯỚC MÔ HÌNH TRÊN 
MAX_EPOCHS = 1 # SÓ LƯỢNG KỶ NGUYÊN TỐI THIỂU CHO HUẤN LUYỆN MÔ HÌNH 
LEARNING_RATE =1e-4 

MODEL_CHECKPOINT = 'bert-base-cased' # Name of pretrained model from 🤗 Model Hub

Laod the WikiText dataset

In [ ]:
from datasets import load_dataset 
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')
print(dataset)

Training a new Tokenizer 

In [ ]:
# lọc ra tất cả dữ liệu train 
# bỏ qua các trường hợp độ dài  = 0 và định dạng xuống dòng trong html 
# từ đó ta có bộ text nối liên tiếp 
all_texts = [
    _ for _ in dataset['train']['test'] if len(_) > 0 and not _.startwith(" =")
]

In [ ]:
def batch_iterator():
    # duyệt qua bộ dự liệu mỗi bước nhảy = 1 lô từ đó lấy ra các lô 
    for i in range(0 , len(all_texts) ,TOKENIZER_BATCH_SIZE ):
        # Sử dụng từ khóa yield để trả về 1 lô văn bản từ danh sách 
        # kích thước mỗi lô = TOKENIZER_BATCH_SIZE 
        yield all_texts[i : i + TOKENIZER_BATCH_SIZE]

In [ ]:
from transformers import AutoTokenizer 
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

tokenizer = tokenizer.train_new_from_iterator(
    batch_iterator() , vocab_size=TOKENIZER_VOCABULARY,
)

Data Pre-processing |

In [ ]:
dataset['train'] = dataset['train'].select([i for i in range(1000)])
dataset['validation'] = dataset['validation'].select([i for i in range(1000)])

In [ ]:
# Implement :
    # 1 :  Chuẩn bị sắn tập dữ liệu cho nhiệm vụ NSP bằng cách tạo các cặp câu (A,B) 
    # trong đó B thực sự theo sau A hoặc B được lấy mẫu ngẫu nhiên từ một nơi khác trong 
    # văn bản .Nó cũng sẽ tạo một bộ nhãn tương ứng cho mỗi cặp , = 1 nếu B thực sự theo sau A else = 0

    # 2 : Mã hóa tập dữ liệu văn bản thành id mã thông báo tương ứng sẽ được sử dụng để nhúng 
    # tra cứu trong bert 

    # 3 : Tạo các  đầu vào bổ sung cho mô hình như token_type_ids , attention_mask 



# Xác định số lượng tokens tối đa sau khi tokenizer (token hóa) mỗi mẫu huấn luyện 
# thực hiện
max_num_tokens = BLOCK_SIZE - tokenizer.num_special_tokens_to_add(pair=True)
# Xây dựng hàm sử dụng huấn luyện các tính năng 

def prepare_train_features(examples):
    """ 
        Function to prepare features for NSP tasl 
    
    Arguments :
        examples : A dictionary with 1 key('text)
            text :  List of raw documents (str)
    Returns:
        examples: A dictionary with 4 keys
        1 Input_ids : List of tokenized , concatnated , and batched 
            sentences from the individual raw documents (int) 
        2 Token_type_ids : List of integers (0 or 1) corresponding 
            to : 0 for sentences A . 1 for sentences B 
        3 Attention_mask : List of integers (0 or 1 ) corresponding
            to: 1 for non-padded tokens, 0 for padded
        
        4 Next_sentence_label: List of integers (0 or 1) corresponding
            to: 1 if the second sentence actually follows the first,
            0 if the senetence is sampled from somewhere else in the corpus
    """

    # Xử lý 
    # xóa đi những mẫu không mong muốn trong tệp huấn luyện 
    examples['document'] = [
        # bỏ đi khoảng trống = strip () và biến đổi các text thành string 
        # sau đó kiểm tra xem chuỗi document có bắt đầu bằng ký tự = hay không 
        d.strip()  for d in examples['text'] if len(d) > 0 and not d.startwith(" =")
    ]
    # Tiếp theo tách cái tài liệu từ tập dữ liệu thành các câu riêng lẻ 
    examples['sentences'] = [
        nltk.tokenize.sent_tokenize(document) for document in examples['document']
    ]

    # Chuyển các tokens trong danh sách các câu thành các id (int) để huấn luyện mô hình 
    examples['tokenized_sequences'] = [
        [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent)) for sent in doc]
        for doc in examples['sentences']
    ]

    # Xác định danh sách các đầu ra gồm 4 danh sách 
    examples ['input_ids'] = [] # int
    examples ['token_type_ids'] = [] # o or 1 
    examples["attention_mask"] = []
    examples["next_sentence_label"] = []

    # Duyệt qua danh sách chứa các câu từ tokenized sentences
    # lấy ra nội dung và chỉ số tương ứng của nó 
    # sử dụng một enumerate như một iterator 
    for doc_index , document in enumerate(examples['tokenized_sentences']):
        # khởi tạo 1 danh sách rỗng current_chunk để lưu trữ các đoạn văn 
        # bản đang được xử lý , mỗi đoạn văn bản là mỗi danh sách các câu 
        # đã được mã hóa thành danh sách các mã thông báo .
        current_chunk = []
        # Khởi tạo biến current_length để lưu trữ độ dài hiện tại của đoạn văn bản đang xử lý 
        # Độ dài được tính bằng số lượng tokens của đoạn văn bản 
        current_length = []
        # khởi tạo biến i = 0 để lưu các đoạn hiện tại trong danh sách 
        i = 0 
        # gán cho độ dài của câu mục tiêu băng với độ dài tối da 
        target_seq_length = max_num_tokens 

        # sử dụng hàm random.random để sinh ra một số ngẫu nhiên [0 -> 1]
        # và so sánh với biến SHORT_SEQ_LENGTH là xác xuất để sinh ra đoạn văn bản ngắn hơn 
        # nếu số ngẫu nhiên nhỏ hơn hơn 0.1  thì  thì target sẽ được gán bằng 
        # một số nguyên ngẫu nhiên từ 2 -> max 
        if random.random() < SHORT_SEQ_PROB:
            target_seq_length = random.randint(2 , max_num_tokens)

        # Bắt đầu vòng while duyệt qua các đoạn trong tài liệu 
        # Thực hiện khối lệnh trong nó khi số đoạn hiện tại < số đoạn trong document 
        while i < len(document):
            # thêm đoạn thứ i của document vào biến segment để chứa các đoạn
            segment = document[i]
            # sau đó thêm đoạn vào danh sách chứa các đoạn hiện tại current_chunk 
            current_chunk.append(segment)
            # Cộng thêm số lượng tokens của đoạn vào độ dài hiện tiện 
            # tức là độ dài được cộng têm 
            target_seq_length += len(segment)

            # kiểm tra xem số đoạn hiện tại có băng với số đoạn trong document - 1 `
            # hay độ dài hiện tại có >= độ dài mục tiêu không 
            # Nếu có thì tiếp tục sử lý các đoạn hiện tại 
            if i == len(document) -1  or current_length >= target_seq_length:
                if current_chunk:
                # Đặt biến a_end là số lượng các đoạn từ danh sách chứa số lượng các đoạn hiện tại sẽ được 
                # sử dụng cho câu A # các đoạn này được lưu trữ trong current_chunk 
                    a_end = 1 
                    # Nếu hiện tại có nhiều hơn 2 đoạn thì 
                    if len(current_chunk) >= 2 :
                        # a_end sẽ được lấy ngẫu nhiên từ 1 -> số lượng các đoạn - 1
                        # tức là 1 -> len(current_chunk) - 1
                        a_end = random.randint(1 , len(current_chunk) - 1)

                    # Khởi tạo danh sách để chứa đựng tokens_a 
                    tokens_a = []
                    # Lặp qua từ 0 đến a_end , a_end cho biết các đoạn hiện tại hiện tại hiện có 
                    # trong current_chunk 
                    for j in range(a_end):
                        # sau đó thêm các tokens trong các đoạn hiện tại vào tokens a
                        # sử dung extend để nối các đoạn 
                        tokens_a.extend(current_chunk[j])
                    
                    # Khởi tại danh sách chứa các tokens B là các tokens sau A
                    tokens_b = []
                    # kiểm tra xem có phải current_chunk có phải chỉ chứa 1 đoạn không 
                    # hay xác xuất sinh ra câu < xác xuất sinh câu tiếp thep NSP_PROB 0.50
                    if len(current_chunk) == 1 or random.random() < NSP_PROB:
                        # Đặt random next = False sẽ được sử dụng để chỉ ra rằng câu B là câu 
                        # ngẫu nhiên chứ không phải là câu tiếp theo của A trong document 
                        # Điều này để tạo ra nhãn cho cac tác vụ NSP
                        is_random_next = True # False 
                        # đặt độ dài mục tiêu của b = độ dìa mục tiêu - độ dài tokens a
                        target_b_length = target_seq_length - len(tokens_a)
                        for _ in range(10):
                            # khởi tạo Random_document để lưu trữ những chỉ số ngẫu nhiên của 
                            # văn bản khác với tài liệu hiện tại từ examples['tokenized_sentences]
                            # sau mỗi lần lặp gán cho random_document_index bộ chỉ số từ 0
                            # đến examples['tokenized_sentences'] - 1
                            random_document_index = random.randint(
                                0 , len(examples['tokenized_sentences']) - 1
                            )
                            # Kiểm tra điều kiện nếu số ngẫu nhiên của câu sinh ra có khác với 
                            # chỉ số của tài liệu hiện tại không có thì thoát 
                            # tức là đoạn mã này sẽ chọn 1 bộ tài liệu bắt kỳ nếu trùng chọn lại 
                        # Mục đích của đoạn mã này để đảm bảo rằng tại liệu ngẫu nhiên không 
                        # Trùng với tài liệu hiện tại đoạn mã sẽ kết thúc khi đạt được kết quả là 
                        # một bộ chỉ số khác nhau tức là khi đạt được mục đích 
                            if random_document_index != doc_index: 
                                break 
                        # Lấy đoạn tài liệu ngẫu nhiên từ examples["tokenized_sentences"] theo các chỉ 
                        # số đã lấy được trước đó random_document_index và gán cho biến random_document
                        random_document = examples['tokenized_sentences'][
                            random_document_index
                        ]
                        # chọn một số ngẫu nhiên 0 -> số lượng đoạn trong tài liệu ngẫu nhiên -1
                        random_start = random.randint(0 , len(random_document) -1)

                        # Lặp các đoạn ngẫu nhiên từ random_start đến hết tài liệu ngẫu nhiên 
                        for j in range(random_start , len(random_document)):
                            # và thêm nó vào danh sách tokens_b 
                            # sử dụng extend để nối các đoạn 
                            tokens_b.extend(random_document[j])
                            # nếu số lượng tokens_b vượt quá độ dài mục tiêu b 
                            if len(tokens_b) >= target_b_length:
                                # dừng lại
                                break
                        # Đếm số lượng các đoạn văn bản không được ử dụng trong danh sách hiện tại 
                        # Giảm giá trị của i đi num_unused_segments để quay lại các đoạn chưa được xử l
                        num_unused_segments = len(current_chunk) - a_end
                        i -= num_unused_segments
                    # Trường hợp còn lại nếu có nhiều hơn một đoạn trong danh sách các đoạn hiện tại 
                    # và xác xuấ NSP_PROB không xảy ra 
                    else: 
                        # Chỉ ra B là câu tiếp theo của A và có nhãn = 1 = True 
                        is_random_next =False # True 
                        # lặp qua danh sách chứa các đoạn hiện tại a_end đến hết số lượng 
                        # các đoạn hiện tại 
                        for j in range(a_end , len(current_chunk)):
                            # Thêm các đoạn hiện tại từ current vào danh sách tokens_ b    
                            tokens_b.extend(current_chunk[j])

                    # Khời tạp đầu vào input_ids (int) 
                    input_ids = tokenizer.build_inputs_with_special_tokens(
                        tokens_a , tokens_b
                    )

                    # Xây dựng bộ tokens_type_ids 0 , 1 
                    token_type_ids = tokenizer.create_token_type_ids_from_sequences(
                        tokens_a, tokens_b
                    )
                    # thêm đệm cho bộ input_ids và token_type_ids 
                    padded = tokenizer.pad(
                        {"input_ids": input_ids, "token_type_ids": token_type_ids},
                        padding="max_length",
                        max_length=MAX_LENGTH,
                    )

                    examples["input_ids"].append(padded["input_ids"])
                    examples["token_type_ids"].append(padded["token_type_ids"])
                    examples["attention_mask"].append(padded["attention_mask"])
                    examples["next_sentence_label"].append(1 if is_random_next else 0)
                    current_chunk = []
                    current_length = 0
            i += 1
    # We delete all the un-necessary columns from our dataset
    del examples["document"]
    del examples["sentences"]
    del examples["text"]
    del examples["tokenized_sentences"]

    return examples

tokenized_dataset= dataset.map(
    prepare_train_features , batched=True , remove_columns=['text'] , num_proc=1
)
               

In [ ]:
from transformers import DataCollatorForLanguageModeling 

collater = DataCollatorForLanguageModeling ( 
    tokenizer=tokenizer , mlm=True , mlm_probability=MLM_PROB, return_tensors='tf'
)

In [ ]:
train = tokenized_dataset['train'].to_tf_dataset(
    columns=['input_ids', 'token_type_ids', 'attention_mask'],
    label_cols =['labels', 'next_sentences_label'], 
    batch_size = TRAIN_BATCH_SIZE, 
    shuffle =True , 
    collate_fn =collater, 
)

validation = tokenized_dataset['validation'].to_tf_dataset(
    columns=['input_ids' , 'token_type_ids'], 
    label_cols =['labels' , 'next_sentences_label'],
    batch_size = TRAIN_BATCH_SIZE , 
    suffle=True , 
    collate_fn=collater, 
)

Defining the model 

In [ ]:
from transformers import BertConfig 
config = BertConfig.from_pretrained(MODEL_CHECKPOINT)

from transformers import TFBertForPreTraining
model = TFBertForPreTraining(config)

optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer)

In [ ]:
model.fit(train, validation_data=validation, epochs=MAX_EPOCHS)

In [ ]:
model.push_to_hub("pretrained-bert", organization="keras-io")
tokenizer.push_to_hub("pretrained-bert", organization="keras-io")

from transformers import TFBertForPreTraining
model = TFBertForPreTraining.from_pretrained("your-username/my-awesome-model")

from transformers import TFBertForSequenceClassification
model = TFBertForSequenceClassification.from_pretrained("your-username/my-awesome-model")